In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('datafiles/preprocessed_water_quality.csv')

#### Declaring GDI and GenSil formulas

In [7]:
from scipy.spatial.distance import cdist
import numpy as np

def generalized_dunn_index(X, labels):
    unique_clusters = np.unique(labels[labels != -1])  # Exclude noise labels
    if len(unique_clusters) < 2:
        return 0

    # Skip if any cluster is empty
    if any(np.sum(labels == k) == 0 for k in unique_clusters):
        return 0

    # Compute centroids
    centroids = np.array([X[labels == k].mean(axis=0) for k in unique_clusters])

    # Intra-cluster distances: max distance from points to their centroid
    intra_dists = [cdist(X[labels == k], [centroids[i]]).max() for i, k in enumerate(unique_clusters)]

    # Inter-cluster distances between centroids
    inter_dists = cdist(centroids, centroids)
    np.fill_diagonal(inter_dists, np.inf)

    denom = np.max(intra_dists)
    if denom == 0:
        return 0

    return np.min(inter_dists) / denom

In [8]:
import numpy as np
from sklearn.metrics import pairwise_distances

def power_mean(arr, p):
    arr = np.array(arr)
    if len(arr) == 0:
        return 0
    if p == 0:
        return np.exp(np.mean(np.log(arr[arr > 0])))  # Geometric mean
    return (np.mean(arr ** p)) ** (1 / p)

def generalized_silhouette_score(X, labels, p=1):
    unique_labels = np.unique(labels)
    if len(unique_labels) < 2:
        return 0

    distances = pairwise_distances(X)
    sil_values = []

    for i in range(len(X)):
        same_cluster = labels == labels[i]
        other_clusters = labels != labels[i]

        a_i = power_mean(distances[i][same_cluster & (np.arange(len(X)) != i)], p)
        b_i = np.min([
            power_mean(distances[i][labels == other_label], p)
            for other_label in unique_labels if other_label != labels[i]
        ])

        if max(a_i, b_i) == 0:
            sil = 0
        else:
            sil = (b_i - a_i) / max(a_i, b_i)
        sil_values.append(sil)

    return np.mean(sil_values)

##### PCA plotting

In [9]:
import plotly.express as px
from sklearn.decomposition import PCA
import pandas as pd

def plot_clusters_3d(X, labels, title="Interactive 3D Cluster Plot"):

    # Reduce to 3 components for 3D plotting
    X_3d = PCA(n_components=3).fit_transform(X)

    # Convert to DataFrame for Plotly
    df_plot = pd.DataFrame(X_3d, columns=["PC1", "PC2", "PC3"])
    df_plot["Cluster"] = labels.astype(str)  # Convert to string for better legend handling

    fig = px.scatter_3d(
        df_plot, x="PC1", y="PC2", z="PC3",
        color="Cluster",
        title=title,
        opacity=0.75,
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    fig.update_layout(
        scene=dict(
            xaxis_title="PCA 1",
            yaxis_title="PCA 2",
            zaxis_title="PCA 3"
        ),
        legend_title_text='Cluster'
    )
    fig.show()

# BIRCH Clustering with Hyperparameter Tuning

In [10]:
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

X = df  

#Hyperparameter grid
thresholds = [0.5, 1.0, 1.5]
branching_factors = [25, 50]
cluster_options = [2, 3, 4, 5]

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing BIRCH hyperparameter tuning...\n")

# Grid search
for t in thresholds:
    for b in branching_factors:
        for k in cluster_options:
            model = Birch(threshold=t, branching_factor=b, n_clusters=k)
            labels = model.fit_predict(X)

            print(f"Trying: threshold={t}, branching_factor={b}, n_clusters={k}, clusters={set(labels)}")

            if len(set(labels)) < 2:
                continue

            try:
                sil = generalized_silhouette_score(X, labels)
                dbi = davies_bouldin_score(X, labels)
                gdi = generalized_dunn_index(X, labels)
                score = sil  # Optimize based on silhouette
            except Exception as e:
                print(f"Error calculating metrics: {e}")
                continue

            if score > best_score:
                best_score = score
                best_params = {'threshold': t, 'branching_factor': b, 'n_clusters': k}
                best_labels = labels
                best_metrics = {'GenSil': sil, 'DBI': dbi, 'GDI': gdi}

# Output
if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics:")
    print(f"Generalized Silhouette Score: {best_metrics['GenSil']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid clustering configuration found. Check data or expand parameter range.")

Performing BIRCH hyperparameter tuning...

Trying: threshold=0.5, branching_factor=25, n_clusters=2, clusters={0, 1}
Trying: threshold=0.5, branching_factor=25, n_clusters=3, clusters={0, 1, 2}
Trying: threshold=0.5, branching_factor=25, n_clusters=4, clusters={0, 1, 2, 3}
Trying: threshold=0.5, branching_factor=25, n_clusters=5, clusters={0, 1, 2, 3, 4}
Trying: threshold=0.5, branching_factor=50, n_clusters=2, clusters={0, 1}
Trying: threshold=0.5, branching_factor=50, n_clusters=3, clusters={0, 1, 2}
Trying: threshold=0.5, branching_factor=50, n_clusters=4, clusters={0, 1, 2, 3}
Trying: threshold=0.5, branching_factor=50, n_clusters=5, clusters={0, 1, 2, 3, 4}
Trying: threshold=1.0, branching_factor=25, n_clusters=2, clusters={0, 1}
Trying: threshold=1.0, branching_factor=25, n_clusters=3, clusters={0, 1, 2}
Trying: threshold=1.0, branching_factor=25, n_clusters=4, clusters={0, 1, 2, 3}
Trying: threshold=1.0, branching_factor=25, n_clusters=5, clusters={0, 1, 2, 3, 4}
Trying: thresho

In [11]:
plot_clusters_3d(df, best_labels, title="BIRCH Clustering with best parameters (Interactive 3D View)")

# Fuzzy C-Means

In [ ]:
from fcmeans import FCM
from sklearn.metrics import davies_bouldin_score
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

X = df  

cluster_options = [2, 3, 4, 5]
fuzziness_values = [1.2, 1.5, 2.0]

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing FCM hyperparameter tuning...\n")

for n_clusters in cluster_options:
    for m in fuzziness_values:
        try:
            model = FCM(n_clusters=n_clusters, m=m, random_state=42)
            model.fit(X.to_numpy())
            labels = model.predict(X.to_numpy())

            if len(set(labels)) < 2:
                continue

            sil = generalized_silhouette_score(X, labels)
            dbi = davies_bouldin_score(X, labels)
            gdi = generalized_dunn_index(X.to_numpy(), labels)
            score = sil

            print(f"Trying: n_clusters={n_clusters}, m={m:.1f}, silhouette={sil:.3f}")

            if score > best_score:
                best_score = score
                best_params = {'n_clusters': n_clusters, 'm': m}
                best_labels = labels
                best_metrics = {'Generalised Silhouette': sil, 'DBI': dbi, 'GDI': gdi}

        except Exception as e:
            print(f"Error during FCM tuning: {e}")
            continue

if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics:")
    print(f"Generalised Silhouette Score: {best_metrics['Generalised Silhouette']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid FCM configuration found. Try expanding parameter range or check input data.")

In [ ]:
plot_clusters_3d(df, best_labels, title="FCM Clustering with best parameters (Interactive 3D View)")

# GMM

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

X = df

# Hyperparameter grid
cluster_options = [2, 3, 4, 5]
covariance_types = ['full', 'tied', 'diag', 'spherical']

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing GMM hyperparameter tuning...\n")

for n_components in cluster_options:
    for cov_type in covariance_types:
        try:
            model = GaussianMixture(n_components=n_components, covariance_type=cov_type, random_state=42)
            model.fit(X)
            labels = model.predict(X)

            if len(set(labels)) < 2:
                continue

            sil = generalized_silhouette_score(X, labels)
            dbi = davies_bouldin_score(X, labels)
            gdi = generalized_dunn_index(X, labels)
            score = sil  # Optimization based on silhouette

            print(f"Trying: n_components={n_components}, cov_type={cov_type}, silhouette={sil:.3f}")

            if score > best_score:
                best_score = score
                best_params = {'n_components': n_components, 'covariance_type': cov_type}
                best_labels = labels
                best_metrics = {'Generalised Silhouette': sil, 'DBI': dbi, 'GDI': gdi}

        except Exception as e:
            print(f"Error during GMM tuning: {e}")
            continue

# Output
if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics:")
    print(f"Generalised Silhouette Score: {best_metrics['Generalised Silhouette']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid GMM configuration found. Try expanding the parameter grid or checking your data.")

In [ ]:
plot_clusters_3d(df, best_labels, title="GMM Clustering with best parameters (Interactive 3D View)")

# K-Meioids

In [ ]:
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import davies_bouldin_score
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

X = df  

# Hyperparameter grid
cluster_options = [2, 3, 4, 5]

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing K-Medoids hyperparameter tuning...\n")

for n_clusters in cluster_options:
    try:
        model = KMedoids(n_clusters=n_clusters, metric='euclidean', random_state=42)
        labels = model.fit_predict(X)

        if len(set(labels)) < 2:
            continue

        sil = generalized_silhouette_score(X, labels)
        dbi = davies_bouldin_score(X, labels)
        gdi = generalized_dunn_index(X, labels)
        score = sil  # Optimization target

        print(f"Trying: n_clusters={n_clusters}, silhouette={sil:.3f}")

        if score > best_score:
            best_score = score
            best_params = {'n_clusters': n_clusters}
            best_labels = labels
            best_metrics = {'Generalised Silhouette': sil, 'DBI': dbi, 'GDI': gdi}

    except Exception as e:
        print(f"Error during K-Medoids tuning: {e}")
        continue

# Output
if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics:")
    print(f"Generalised Silhouette Score: {best_metrics['Generalised Silhouette']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid K-Medoids configuration found. Try expanding parameter range or check input data.")


In [ ]:
plot_clusters_3d(df, best_labels, title="K-Medoids Clustering with best parameters (Interactive 3D View)")

# OPTICS

In [6]:
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

X = df  

# Hyperparameter grid for OPTICS
min_samples_list = [5, 10, 15]
xi_values = [0.01, 0.05, 0.1]

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing OPTICS hyperparameter tuning...\n")

for min_samples in min_samples_list:
    for xi in xi_values:
        model = OPTICS(min_samples=min_samples, xi=xi)
        labels = model.fit_predict(X)

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        print(f"Trying: min_samples={min_samples}, xi={xi}, clusters={n_clusters}, labels={set(labels)}")

        if n_clusters < 2:
            continue

        try:
            sil = generalized_silhouette_score(X[labels != -1], labels[labels != -1])  # exclude noise
            dbi = davies_bouldin_score(X[labels != -1], labels[labels != -1])
            gdi = generalized_dunn_index(X, labels)
            score = sil  
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            continue

        if score > best_score:
            best_score = score
            best_params = {'min_samples': min_samples, 'xi': xi}
            best_labels = labels
            best_metrics = {'Generalised Silhouette': sil, 'DBI': dbi, 'GDI': gdi}

# Output
if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics (excluding noise points for Sil & DBI):")
    print(f"Generalised Silhouette Score: {best_metrics['Generalised Silhouette']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid clustering configuration found. Check data or expand parameter range.")

Performing OPTICS hyperparameter tuning...

Trying: min_samples=5, xi=0.01, clusters=2447, labels={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 2

In [7]:
plot_clusters_3d(df, best_labels, title="OPTICS Clustering with best parameters (Interactive 3D View)")

# SOM

In [ ]:
from minisom import MiniSom
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
from collections import defaultdict

X = df  

# SOM grid search settings
grid_sizes = [(2, 2), (3, 3), (4, 4)]
iterations = 1000

best_score = -1
best_params = {}
best_labels = None
best_metrics = {}

print("Performing SOM hyperparameter tuning...\n")

for x_dim, y_dim in grid_sizes:
    try:
        som = MiniSom(x=x_dim, y=y_dim, input_len=X.shape[1], sigma=1.0, learning_rate=0.5,
                      neighborhood_function='gaussian', random_seed=42)
        som.train_random(X, iterations)

        # Assign clusters based on winning neuron positions
        win_map = defaultdict(list)
        labels = []

        for i, x in enumerate(X):
            w = som.winner(x)
            label = w[0] * y_dim + w[1]  # Convert 2D neuron location to unique label
            labels.append(label)

        labels = np.array(labels)

        if len(set(labels)) < 2:
            continue

        sil = generalized_silhouette_score(X, labels)
        dbi = davies_bouldin_score(X, labels)
        gdi = generalized_dunn_index(X, labels)
        score = sil  # Optimization target

        print(f"Trying: SOM {x_dim}x{y_dim}, Generalised Silhouette={sil:.3f}")

        if score > best_score:
            best_score = score
            best_params = {'grid_size': f'{x_dim}x{y_dim}'}
            best_labels = labels
            best_metrics = {'Generalised Silhouette': sil, 'DBI': dbi, 'GDI': gdi}

    except Exception as e:
        print(f"Error during SOM training: {e}")
        continue

# Output
if best_metrics:
    print("\n✅ Best Parameters Found:")
    for key, val in best_params.items():
        print(f"  {key}: {val}")

    print("\n📊 Evaluation Metrics:")
    print(f"Generalised Silhouette Score: {best_metrics['Generalised Silhouette']:.3f}")
    print(f"Davies-Bouldin Index: {best_metrics['DBI']:.3f}")
    print(f"Generalized Dunn Index: {best_metrics['GDI']:.3f}")

    print("\n📌 Cluster Distribution:")
    print(pd.Series(best_labels).value_counts().sort_index())

else:
    print("❌ No valid SOM configuration found. Try expanding grid sizes or check input data.")

In [ ]:
plot_clusters_3d(df, best_labels, title="SOM Clustering with best parameters (Interactive 3D View)")